# Absorption and Multiple Scattering Correction

## Introduction

### General aspects

When neutrons travel into the sample, there are multiple types of events that can happen, and two typical ones are scattering and absorption. Specifically concerning the scattering event, there are also two types -- single scattering and multiple scattering. Fundamentally, it is only the single scattering event that we are concerned about for extracting structural information, in a general sense. Therefore, in practice, we need to correct for both the absorption and multiple scattering effects. With this regard, the Mantid concept page as in Ref. presents a nice summary about the undelying principles and the practical implementation in Mantid framework. On top of that, the derivation of the Paalman-Pings scheme for the absorption correction is detailed below. First, the complete expression of the full Paalman-Pings formulation is as follows,

$$\begin{align}I_{S+C}^{E} = I_{S}A_{S,SC} + I_{C}A_{C,SC} + I_{SC}A_{SC,SC}\tag{1}\end{align}$$

where,

- $S$ for sample, $C$ for container

- $I_{S+C}^{E}$ experimentally measured intensity from $S+C$.

- $I_S$ theoretical intensity from the isolated sample.

- $I_C$ theoretical intensity from the isolated container.

- $I_{SC}$ theoretical intensity from the correlated sample and container interface.

- $A_{S,SC}$ is the absorption factor for scattering in the sample region and absorption by the sample and container.

- $A_{C,SC}$ is the absorption factor for scattering in the container region and absorption by the sample and container.

- $A_{AC,SC}$ is the absorption factor for scattering in the correlated sample and container interface and absorption by the sample and container.

Ignoring the contribution from the correlated interface between the sample and container, the exression can be simplified to,

$$\begin{align}I_{S+C}^{E} = I_{S}A_{S,SC} + I_{C}A_{C,SC}\tag{2}\end{align}$$

For the empty container measurement, one has,

$$I_C^E = I_CA_{C,C} \Rightarrow I_C = \frac{1}{A_{C,C}}I_C^E\tag{3}$$

where,

- $I_C^E$ experimentally measured intensity from $S+C$experimentally measured intensity from $C$.

- $A_{C,C}$ is the absorption factor for scattering in the container region and absorption by the container.

Putting Eqn. (2) and (3) together, one could obtain,

$$I_{S+C}^E = I_SA_{S,SC} + \frac{A_{C,SC}}{A_{C,C}}I_C^E \Rightarrow I_S = \frac{1}{A_{S,SC}}I_{S+C}^E - \frac{A_{C,SC}}{A_{S,SC}}\frac{1}{A_{C,C}}I_C^E$$

Following such a scheme, the absorption coefficients (i.e., those $A$'s) can be evaluated in a numerical integration approach and applied to the experimentally measured intensities to correct for the absorption effect as detailed above.

## Notes on Multiple Scattering Correction Implementation

- Put down the contents in the overleaf doc located at [https://latex.ornl.gov/project/6144f47987c0cd00b83846d9](https://latex.ornl.gov/project/6144f47987c0cd00b83846d9)